# Predicting Power Outage Durations

## Code

In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

import plotly.express as px
pd.options.plotting.backend = 'plotly'

### Framing the Problem

Major power outages can have detrimental effects on public safety, infrastructure, and the economy. Therefore, predicting power outage severity is crucial for proactive planning and resource allocation. It enables utilities and emergency services to allocate resources efficiently and mitigate potential damages, ultimately reducing the impact on the community. We seek to anticipate power outage severity by predicting outage durations, since longer power outages tend to be more negatively influential. Given that a power outage just occurred, we want to know how long the outage will last depending on the cause of the power outage, as well as population metrics, cost metrics, season, geography, and time. We will be implementing a k-Nearest Neighbors (k-NN) regressor that incorporates these variables to predict outage duration. In order to evaluate our model for its accuracy, we plan to use RMSE and R-squared as our metrics. We hope to minimize RMSE while maximizing R-squared.

#### Cleaning and EDA

First, let's take a look at our dataset:

In [261]:
outages = pd.read_excel("outage.xlsx")
outages.head()

,variables,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,...,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
0,Units,NaN,NaN,NaN,NaN,NaN,NaN,NaN,numeric,NaN,...,%,%,persons per square mile,persons per square mile,persons per square mile,%,%,%,%,%
1,NaN,1.0,2011.0,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
2,NaN,2.0,2014.0,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
3,NaN,3.0,2010.0,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
4,NaN,4.0,2012.0,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743


At first glance, we can see that the first two columns and the first row of the dataset include many NaN values. This is because the variable column denotes the units used for each measure in the following columns, as described in the first row. We will remove these since we already have information about the units of each variable.

In [262]:
# drop the first row
outages = outages.drop(0)
outages = outages.reset_index(drop=True)
# drop first two columns
outages = outages.drop(columns=['variables', "OBS"])

We also want to combine the OUTAGE.START.TIME and OUTAGE.START.DATE columns into one variable called OUTAGE.START. We will do so using some helper functions.

In [263]:
# convert columns to string, obtain the relevant information, combine the two strings (taking into 
# account the null values), and convert string to a datetime object
def combine_times(df, col1, col2):
    combine = df[col1].astype(str).str[:11] + df[col2].astype(str)
    return pd.to_datetime(combine.apply(lambda x: np.nan if x == 'nannan' else x), format='%Y-%m-%d %H:%M:%S')

In [264]:
outages = outages.assign(**{'OUTAGE.START':combine_times(outages, "OUTAGE.START.DATE", "OUTAGE.START.TIME")})

Since we are only interested in predicting outage durations based on population, cost, geography, time, and causes of the power outage, we will only keep the appropriate columns. The columns we wanted to focus on are OUTAGE.DURATION, YEAR, MONTH, U.S.STATE, CAUSE.CATEGORY, and, for now, all of the quantitative variables in the dataset that represent population and cost metrics. 

In [265]:
outages = outages.drop(columns=outages.columns[3:4])
outages = outages.drop(columns=["CAUSE.CATEGORY.DETAIL", "HURRICANE.NAMES", "DEMAND.LOSS.MW", 
                                "CUSTOMERS.AFFECTED", "NERC.REGION", "CLIMATE.REGION", 
                                "CLIMATE.CATEGORY", "OUTAGE.START.DATE", "OUTAGE.START.TIME", 
                               "OUTAGE.RESTORATION.DATE", "OUTAGE.RESTORATION.TIME"])

Now let's take a look at our sub-dataset. Do we need to do any additional data cleaning?

In [266]:
outages.head()

,YEAR,MONTH,U.S._STATE,ANOMALY.LEVEL,CAUSE.CATEGORY,OUTAGE.DURATION,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,...,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START
0,2011.0,7.0,Minnesota,-0.3,severe weather,3060,11.6,9.18,6.81,9.28,...,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2011-07-01 17:00:00
1,2014.0,5.0,Minnesota,-0.1,intentional attack,1,12.12,9.71,6.49,9.28,...,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2014-05-11 18:38:00
2,2010.0,10.0,Minnesota,-1.5,severe weather,3000,10.87,8.19,6.07,8.15,...,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2010-10-26 20:00:00
3,2012.0,6.0,Minnesota,-0.1,severe weather,2550,11.79,9.25,6.71,9.19,...,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2012-06-19 04:30:00
4,2015.0,7.0,Minnesota,1.2,severe weather,1740,13.07,10.16,7.74,10.43,...,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2015-07-18 02:00:00


In [267]:
outages.dtypes

YEAR                        float64
MONTH                       float64
U.S._STATE                   object
ANOMALY.LEVEL                object
CAUSE.CATEGORY               object
OUTAGE.DURATION              object
RES.PRICE                    object
COM.PRICE                    object
IND.PRICE                    object
TOTAL.PRICE                  object
RES.SALES                    object
COM.SALES                    object
IND.SALES                    object
TOTAL.SALES                  object
RES.PERCEN                   object
COM.PERCEN                   object
IND.PERCEN                   object
RES.CUSTOMERS               float64
COM.CUSTOMERS               float64
IND.CUSTOMERS               float64
TOTAL.CUSTOMERS             float64
RES.CUST.PCT                 object
COM.CUST.PCT                 object
IND.CUST.PCT                 object
PC.REALGSP.STATE             object
PC.REALGSP.USA               object
PC.REALGSP.REL               object
PC.REALGSP.CHANGE           

We need to convert all of the quantitative variables into float data types, as they are currently stored as object data types.

In [268]:
for i in outages.columns[5:-1]:
    outages = outages.assign(**{i:outages[i].astype(float)})
outages = outages.assign(**{'ANOMALY.LEVEL':outages['ANOMALY.LEVEL'].astype(float)})

Great! Now we have a clean dataset we can work with to identify possible relationships between variables and thereafter predict outage duration using said variables.

In [269]:
outages.dtypes

YEAR                        float64
MONTH                       float64
U.S._STATE                   object
ANOMALY.LEVEL               float64
CAUSE.CATEGORY               object
OUTAGE.DURATION             float64
RES.PRICE                   float64
COM.PRICE                   float64
IND.PRICE                   float64
TOTAL.PRICE                 float64
RES.SALES                   float64
COM.SALES                   float64
IND.SALES                   float64
TOTAL.SALES                 float64
RES.PERCEN                  float64
COM.PERCEN                  float64
IND.PERCEN                  float64
RES.CUSTOMERS               float64
COM.CUSTOMERS               float64
IND.CUSTOMERS               float64
TOTAL.CUSTOMERS             float64
RES.CUST.PCT                float64
COM.CUST.PCT                float64
IND.CUST.PCT                float64
PC.REALGSP.STATE            float64
PC.REALGSP.USA              float64
PC.REALGSP.REL              float64
PC.REALGSP.CHANGE           

But wait! We also saw that season significantly impacted outage duration times in our data exploration project on power outages. This may be a useful metric for our regression model later on, so we have created a helper function that takes in the month of the power outage and returns the season that the power outage occurred in. This was saved to a new column called SEASON.

In [270]:
def season(month):
    if month in [12.0, 1.0, 2.0]:
        return "winter"
    elif month in [3.0, 4.0, 5.0]:
        return "spring"
    elif month in [6.0, 7.0, 8.0]:
        return "summer"
    elif month in [9.0, 10.0, 11.0]:
        return "fall"
    else:
        return np.NaN

In [271]:
outages = outages.assign(SEASON=outages["MONTH"].apply(season))
outages.head()

,YEAR,MONTH,U.S._STATE,ANOMALY.LEVEL,CAUSE.CATEGORY,OUTAGE.DURATION,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,...,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START,SEASON
0,2011.0,7.0,Minnesota,-0.3,severe weather,3060.0,11.60,9.18,6.81,9.28,...,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2011-07-01 17:00:00,summer
1,2014.0,5.0,Minnesota,-0.1,intentional attack,1.0,12.12,9.71,6.49,9.28,...,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2014-05-11 18:38:00,spring
2,2010.0,10.0,Minnesota,-1.5,severe weather,3000.0,10.87,8.19,6.07,8.15,...,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2010-10-26 20:00:00,fall
3,2012.0,6.0,Minnesota,-0.1,severe weather,2550.0,11.79,9.25,6.71,9.19,...,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2012-06-19 04:30:00,summer
4,2015.0,7.0,Minnesota,1.2,severe weather,1740.0,13.07,10.16,7.74,10.43,...,2279.0,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2015-07-18 02:00:00,summer


Perfect! Now we can get started on exploring the data.

### Baseline Model

What can we expect in terms of outage duration in general? Will the values be high? Will there be a lot of variation? Let's investigate the distribution of power outage durations by plotting a boxplot of outage durations.

In [27]:
fig1 = px.box(outages, y="OUTAGE.DURATION")
fig1.show()

Wow! There seems to be a lot of outliers in our dataset. The boxplot appears small compared to the rest of our graph because the mean outage duration is very far from the max outage duration. This could negatively influence our prediction model, since outliers could skew our prediction towards higher values, even though most of our data centers around a lower threshold. In order to ensure that our prediction model follows the trend of the majority of our data, and that it doesn't become too biased from outlier datapoints, we will only look at power outages that are within the upper fence of the boxplot, which is 7020 minutes.

In [28]:
outages.shape

(1534, 45)

In [272]:
outages = outages[outages["OUTAGE.DURATION"] <= 7020]
outages.shape

(1332, 45)

We still have a good amount of data to work with. As we noticed in the boxplot, only a small amount of our data is above 7020 minutes.

In [30]:
outages["OUTAGE.DURATION"].isna().sum()

0

We also don't have any null values, so we can work with our new dataset directly without having to drop or impute null values.

Since we are using a regression model to predict outage duration times, we need to find variables that can act as good predictors for our dependent variable. We have an abundant list of quantitative variables we could use to predict outage duration times, so let's see if we can find any correlations between each variable and outage duration.  

Below we generated scatter plots of each quantitative variable with outage duration times. While there are only 3 graphs illustrated below, we went through all of the graphs in increments of 5 and looked for any possible relationships between outage durations and other quantitative variables.

In [31]:
for i in outages.columns[15:18]:
    plot = px.scatter(x=outages[i].astype(float), y=outages["OUTAGE.DURATION"].astype(float), title=i,
                     labels={'x': i, 'y':'OUTAGE.DURATION'})
    plot.show()

Unfortunately it doesn't look like there are any clear relationships between duration times and the other quantitative variables, such as COM.CUSTOMERS. Most of the points seem to be clustered. Some graphs did look more promising than others, including RES.CUST.PCT, COM.CUST.PCT, IND.CUST.PCT, PC.REALGSP.USA, UTIL.CONTRI, POPPCT_UC, POPDEN_RURAL, PCT_LAND, and PCT_WATER_TOT. To get a closer look, let's see what the correlation coefficents are between outage duration and each quantitative variable.

In [32]:
quant_values = outages[outages.columns[4:-2]]
quant_values.corr().loc["OUTAGE.DURATION"]

OUTAGE.DURATION      1.000000
RES.PRICE           -0.062971
COM.PRICE           -0.062640
IND.PRICE           -0.068729
TOTAL.PRICE         -0.064647
RES.SALES            0.020440
COM.SALES           -0.001665
IND.SALES            0.053275
TOTAL.SALES          0.021107
RES.PERCEN           0.001520
COM.PERCEN          -0.042720
IND.PERCEN           0.031711
RES.CUSTOMERS        0.005632
COM.CUSTOMERS       -0.003636
IND.CUSTOMERS       -0.079793
TOTAL.CUSTOMERS      0.003765
RES.CUST.PCT         0.151205
COM.CUST.PCT        -0.120901
IND.CUST.PCT        -0.142698
PC.REALGSP.STATE    -0.069015
PC.REALGSP.USA      -0.157613
PC.REALGSP.REL      -0.052926
PC.REALGSP.CHANGE   -0.006979
UTIL.REALGSP         0.016917
TOTAL.REALGSP       -0.033574
UTIL.CONTRI          0.147114
PI.UTIL.OFUSA        0.011527
POPULATION          -0.014532
POPPCT_URBAN        -0.041972
POPPCT_UC           -0.120078
POPDEN_URBAN        -0.049145
POPDEN_UC           -0.080384
POPDEN_RURAL         0.134183
AREAPCT_UR

The correlation coefficients seem to fit our interpretation of the graphs. Yet, these variables have relatively low correlation with outage duration, the highest being only 0.266. As such, our quantitative variables may not be good linear predictors for outage duration. We will further explore this in our final model, but for now let's focus on categorical variables, as they may tell us more about outage duration times.

Intuitively, a strong predictor for outage duration may be the cause of the power outage. In our analysis, we are assuming that the power outage has just occurred, and that the cause of the outage is known. Based on this information, we may be able to generalize outage duration times. For example, severe-weather induced power outages may last longer than intentional attacks since there is a higher sense of urgency for attacks, and because most of the time companies must wait for the harsh weather conditions to pass. Another parameter that could significantly influence power outage durations is location. Different states may have different weather patterns, population sizes, availability of resources, local regulations, and so on. Accordingly, for our baseline model we focus on these two predictors, CAUSE.CATEGORY and U.S._STATE. These variables are both nominal. CAUSE,CATEGORY depicts the cause of the power outage, like severe-weather or intentional attack, while U.S._STATE depicts the state where the power outage occurred, like California. We are one-hot encoding both variables so we can input them into our KNN regression model.

In [273]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(outages.drop(columns=["OUTAGE.DURATION"]), outages["OUTAGE.DURATION"], 
                                                        test_size=0.20)

In [274]:
def knn_reg_perf(galton, X_train, X_test, y_train, y_test):

    from sklearn.neighbors import KNeighborsRegressor
    
    def rmse(actual, pred):
        return np.sqrt(np.mean((actual - pred) ** 2))
    
    train_rmse_error = []
    test_rmse_error = []
    train_r2_error = []
    test_r2_error = []
    
    
    for n in range(1,101):
        
        preproc = ColumnTransformer(
            transformers=[
                ('group', OneHotEncoder(handle_unknown='ignore'), ["U.S._STATE", "CAUSE.CATEGORY"]),
            ],
            remainder='drop'
        )
        
        neigh = Pipeline([
            ('preprocessor', preproc), 
            ("decision-tree", KNeighborsRegressor(n_neighbors=n))
        ])
        
        neigh.fit(X_train, y_train)
    
        train_rmse_error.append(rmse(y_train, neigh.predict(X_train)))
        train_r2_error.append(neigh.score(X_train, y_train))
        test_rmse_error.append(rmse(y_test, neigh.predict(X_test)))
        test_r2_error.append(neigh.score(X_test, y_test))
    
    return pd.DataFrame({'train_rmse_err': train_rmse_error, "test_rmse_err": test_rmse_error, 
                         'train_r2_err': train_r2_error, "test_r2_err": test_r2_error}, index=(range(1, 101)))

In [275]:
knn_baseline = knn_reg_perf(outages, X_train, X_test, y_train, y_test)
knn_baseline

,train_rmse_err,test_rmse_err,train_r2_err,test_r2_err
1,1702.724622,1666.484082,-0.093349,0.053290
2,1359.957759,1398.981824,0.302538,0.332826
3,1330.505409,1383.420338,0.332420,0.347586
4,1312.098308,1383.492993,0.350764,0.347518
5,1302.786034,1354.160034,0.359947,0.374893
...,...,...,...,...
96,1343.156410,1340.398933,0.319664,0.387533
97,1343.099580,1339.487606,0.319722,0.388365
98,1344.285945,1340.630712,0.318520,0.387321
99,1344.776812,1340.550310,0.318022,0.387394


The output of our knn_reg_perf function is a dataframe of training and testing rmse and r-squared values from 1 to 200. The index of the dataframe denotes the number of neighbors used in the regressor. We plot this below to get a better view of our error data.

In [276]:
fig4 = px.line(x=range(1,101), y=[knn_baseline['train_rmse_err'], knn_baseline['test_rmse_err']], 
               labels={'value': 'RMSE', "x": "Number of Neighbors"},
              title='Average RMSE', line_shape='linear')
newnames = {'wide_variable_0':'Train', 'wide_variable_1': 'Test'}
fig4.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
# Show the plot
fig4.show()

Our rmse for both training and testing appears to be minimized around 20. What does our r-squared look like?

In [277]:
fig5 = px.line(x=range(1,101), y=[knn_baseline['train_r2_err'], knn_baseline['test_r2_err']], 
               labels={'value': 'R-squared', "x": "Number of Neighbors"},
              title='Average R-squared', line_shape='linear')
newnames = {'wide_variable_0':'Train', 'wide_variable_1': 'Test'}
fig5.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
# Show the plot
fig5.show()

Luckily for us, both our training and test r-squared errors appear to be maximized at around 10. To get a more accurate number for our neighbor metric we will use gridsearchcv.

After using gridsearch we see that 10 is our optimized parameter for number of neighbors. Below is the data for our rmse and r-2 squared values for testing and training with 10 neighbors using a KNN model.

In [279]:
knn_baseline.iloc[10]

train_rmse_err    1262.896470
test_rmse_err     1302.577787
train_r2_err         0.398542
test_r2_err          0.421608
Name: 11, dtype: float64

As we can see, our rmse is about 1200 minutes, which is about 16 hours. Additionally, our r-squared values are around .40, which isn't very high. While our rmse is high, we have a lot of variation in our outage duration data that could explain this score. We believe that for a simple KNN regression model these scores are not bad, but we hope to improve them in our final model by adding more features.

## Final Model

For our final model, we want to try to improve upon the baseline model. We will continue to one-hot encode U.S.STATE and CAUSE.CATEGORY. Additionally, we decided to feature engineer two new features, the time of the outage, as well as one of the quantitative features we explored previously. To better predict power outages based on time, we looked at peak hours for energy consumption, which tends to be from 4pm to 9pm. We expect that power outages that started during peak hours would impact the severity of the power outage. In order to include this in our KNN regression model, we transformed our OUTAGE.START column into 0 or 1 for peak vs non-peak hours. We utilized a function transformer, as well as a helper function, to do this. For our other additional feature, we need to look at possible combinations of MONTH and YEAR with the quantitative variables that had the highest correlation coefficient with outage duration. In order to identify an optimized combination, we will perform a manual iterative method that finds the average rmse score for each KNN regressor that includes said combination. For example, in our first iteration, we will employ a stdscalarbygroup transformer on MONTH and RES.CUST.PCT, where MONTH is the month that the power outage occurred in and RES.CUST.PCT is the percent of residential customers served in the U.S. state in percentage. The stdscalarbygroup transformer standardizes the quantitative variable by grouping the categorical variable. This means that the RES.CUST.PCT would be standardized based on the month. We have created a function that goes through each combination, finds the average rmse for 10 different k neighbors, and returns a dictionary with this information that we can then use to pick the best parameters for our stdscalarbygroup transformer. 

In [280]:
from sklearn.base import BaseEstimator, TransformerMixin

class StdScalerByGroup(BaseEstimator, TransformerMixin):

    def __init__(self):
        self.grps_ = None

    def fit(self, X, y=None):
        # X might not be a pandas DataFrame (e.g. a np.array)
        df = pd.DataFrame(X)

        # Compute and store the means/standard-deviations for each column (e.g. 'c1' and 'c2'), 
        # for each group (e.g. 'A', 'B', 'C').  
        # (Our solution uses a dictionary)
        
        groups = df.iloc[:, 0]
        quant_data = df.iloc[:, 1:]

        grps = {}

        for group_name, group_values in quant_data.groupby(groups):
            mean = group_values.mean()
            std = group_values.std()

            grps[group_name] = {'mean': mean, 'std': std}
     
        self.grps_ = grps

        return self

    def transform(self, X, y=None):

        try:
            getattr(self, "grps_")
        except AttributeError:
            raise RuntimeError("You must fit the transformer before tranforming the data!")

        df = pd.DataFrame(X)

        groups = df.iloc[:, 0]
        quant_data = df.iloc[:, 1:]

        new_data = []

        for group_name, group_values in quant_data.groupby(groups):
            mean = self.grps_[group_name]['mean']
            std = self.grps_[group_name]['std']

            standardize = (group_values - mean) / std
            new_data.append(standardize)
        
        transformed_df = pd.concat(new_data)
        transformed_df.index = groups.values
        return transformed_df

In [281]:
def knn_reg_perf(galton, X_train, X_test, y_train, y_test):
    
    from sklearn.neighbors import KNeighborsRegressor
    
    def rmse(actual, pred):
        return np.sqrt(np.mean((actual - pred) ** 2))
    
    def peak_hours(x):
        df = pd.DataFrame((x["OUTAGE.START"].dt.hour >= 16) & (x["OUTAGE.START"].dt.hour <= 21))
        return df.replace({True: 1, False: 0})
    
    train_rmse_error = []
    test_rmse_error = []
    train_r2_error = []
    test_r2_error = []
    
    category = ["MONTH", "YEAR"]
    quantitative = ["RES.CUST.PCT", "COM.CUST.PCT", "IND.CUST.PCT", 
                    "UTIL.CONTRI", "POPPCT_UC", 
                    "PCT_LAND", "PCT_WATER_TOT"]
    
    parameter_dict = {}
    
    for i in category:
        
        X_train
        for j in quantitative:
            train_rmse_error = []
            test_rmse_error = []
            for n in range(1,11):
        
                preproc = ColumnTransformer(
                    transformers=[
                        ('peak_hours', FunctionTransformer(peak_hours), ["OUTAGE.START"]),
                        ('standardize_pop', StdScalerByGroup(), [i, j]),
                        ('group', OneHotEncoder(handle_unknown='ignore'), ["MONTH", "U.S._STATE", "CAUSE.CATEGORY"]),
                    ],
                    remainder='drop'
                )
        
                neigh = Pipeline([
                    ('preprocessor', preproc), 
                    ("decision-tree", KNeighborsRegressor(n_neighbors=n))
                ])
        
                neigh.fit(X_train, y_train)
    
                train_rmse_error.append(rmse(y_train, neigh.predict(X_train)))
                test_rmse_error.append(rmse(y_test, neigh.predict(X_test)))
            
            parameter_dict[i + " and " + j] = np.mean(train_rmse_error)
    
    return parameter_dict

In [282]:
knn_reg_perf(outages, X_train, X_test, y_train, y_test)

{'MONTH and RES.CUST.PCT': 1041.8175662539786,
 'MONTH and COM.CUST.PCT': 1050.6551061980074,
 'MONTH and IND.CUST.PCT': 1049.2647862810243,
 'MONTH and UTIL.CONTRI': 1064.8836904307304,
 'MONTH and POPPCT_UC': 1055.798922414368,
 'MONTH and PCT_LAND': 1047.8665686158743,
 'MONTH and PCT_WATER_TOT': 1047.8737272062413,
 'YEAR and RES.CUST.PCT': 1053.6275358286316,
 'YEAR and COM.CUST.PCT': 1059.9977452565192,
 'YEAR and IND.CUST.PCT': 1043.4997138594188,
 'YEAR and UTIL.CONTRI': 1059.834964018724,
 'YEAR and POPPCT_UC': 1021.0970047450497,
 'YEAR and PCT_LAND': 1029.7555421467164,
 'YEAR and PCT_WATER_TOT': 1029.3621801240333}

We see that the lowest rmse score is YEAR and POPPCT_UC, where POPPCT_UC is percentage of the total population of the U.S. state represented by the population of the urban clusters. We will therefore incorporate this into our final baseline model. 

In [283]:
def knn_reg_perf(galton, X_train, X_test, y_train, y_test):
    
    from sklearn.neighbors import KNeighborsRegressor
    
    def rmse(actual, pred):
        return np.sqrt(np.mean((actual - pred) ** 2))
    
    def peak_hours(x):
        df = pd.DataFrame((x["OUTAGE.START"].dt.hour >= 16) & (x["OUTAGE.START"].dt.hour <= 21))
        return df.replace({True: 1, False: 0})
    
    train_rmse_error = []
    test_rmse_error = []
    train_r2_error = []
    test_r2_error = []
    
    for n in range(1,51):
        
        preproc = ColumnTransformer(
            transformers=[
                ('peak_hours', FunctionTransformer(peak_hours), ["OUTAGE.START"]),
                ('standardize_pop', StdScalerByGroup(), ["YEAR", "POPPCT_UC"]),
                ('group', OneHotEncoder(handle_unknown='ignore'), ["MONTH", "U.S._STATE", "CAUSE.CATEGORY"]),
            ],
            remainder='drop'
        )
        
        neigh = Pipeline([
            ('preprocessor', preproc), 
            ("decision-tree", KNeighborsRegressor(n_neighbors=n))
        ])
        
        neigh.fit(X_train, y_train)
    
        train_rmse_error.append(rmse(y_train, neigh.predict(X_train)))
        train_r2_error.append(np.var(neigh.predict(X_train)) / np.var(y_train))
        test_rmse_error.append(rmse(y_test, neigh.predict(X_test)))
        test_r2_error.append(np.var(neigh.predict(X_test)) / np.var(y_test))
    
    return pd.DataFrame({'train_rmse_err': train_rmse_error, "test_rmse_err": test_rmse_error, 
                         'train_r2_err': train_r2_error, "test_r2_err": test_r2_error}, index=(range(1,51)))

In [284]:
knn_final = knn_reg_perf(outages, X_train, X_test, y_train, y_test)

In [285]:
knn_final

,train_rmse_err,test_rmse_err,train_r2_err,test_r2_err
1,0.000000,2026.578199,1.000000,1.096387
2,882.574040,1714.307178,0.742488,0.732466
3,1012.346238,1592.008311,0.616704,0.631154
4,1086.578248,1527.627324,0.522254,0.533778
5,1133.639676,1452.506661,0.469748,0.461680
6,1178.082975,1427.952159,0.436228,0.421373
7,1210.244199,1418.709481,0.407694,0.388674
8,1216.044152,1401.111948,0.388287,0.366430
9,1239.693887,1403.126365,0.362091,0.358780
10,1251.766633,1376.575739,0.344443,0.357184


We can graph the output of our rmse and r-squared scores below, like we did for our baseline.

In [286]:
fig6 = px.line(x=range(1,51), y=[knn_final['train_rmse_err'], knn_final['test_rmse_err']], 
               labels={'value': 'RMSE', "x": "Number of Neighbors"},
              title='Average RMSE', line_shape='linear')
newnames = {'wide_variable_0':'Train', 'wide_variable_1': 'Test'}
fig6.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
# Show the plot
fig6.show()

In [287]:
fig7 = px.line(x=range(1,51), y=[knn_final['train_r2_err'], knn_final['test_r2_err']], 
               labels={'value': 'R-squared', "x": "Number of Neighbors"},
              title='Average R-squared', line_shape='linear')
newnames = {'wide_variable_0':'Train', 'wide_variable_1': 'Test'}
fig7.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
# Show the plot
fig7.show()

Based on these graphs, it seems that rmse is minimized at the beginning, and r-squared is similarily maximized at the beginning. We will therefore use a parameter of 2 for the number of neighbors.

In [290]:
knn_final.iloc[1]

train_rmse_err     882.574040
test_rmse_err     1714.307178
train_r2_err         0.742488
test_r2_err          0.732466
Name: 2, dtype: float64

Our training rmse score is 882, our testing rmse score is 1714, our training r-squared score is 0.74, and our testing r-squared score is .73. As we can see the training rmse score decreased from our baseline and our r-squared scores for both the training and test sets improved by about 0.30. However, our rmse score for the testing set increased slightly. We may be able to improve our model to better fit unknown data with other parameters. We believe that we were able to achieve better training rmse because we have added more features to our model that can help predict outage duration time. More specifically, peak energy consumption hours can impact severity of a power outage since resources are exhausted. Similarily, year and U.S. state percentage population can inform us about possible power outage durations, as some years may have more outages and higher populations could lead to longer durations, as more people are utilizing energy. Lastly, as we saw in our baseline, the location and cause of the category is a strong indicator of outage duration. This makes sense, as most power outages in our dataset are severe-weather induced, meaning that other variables like cost or population don't have great influence over duration time, since the most important attribute is how long the weather event lasts.

### Fairness Analysis

In [ ]:
# TODO